# CLARISSA Portal: GitLab API Playground

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/wolfram-laube/clarissa/blob/main/docs/tutorials/notebooks/12_GitLab_API_Playground.ipynb)
[![GitLab](https://img.shields.io/badge/GitLab-Repository-orange)](https://gitlab.com/wolfram_laube/blauweiss_llc/irena)

**Interactive playground for GitLab project management operations**

This notebook provides hands-on access to GitLab API operations for the CLARISSA project:

- 📋 **Issues** - Create, list, update, close issues
- 🔀 **Merge Requests** - Create MRs, review, merge
- 🚀 **Pipelines** - Trigger, monitor, cancel CI/CD jobs
- 🏷️ **Labels & Milestones** - Project organization
- 📁 **Repository** - Files, branches, commits

---

## ⚠️ Security Notice

This notebook uses the project's GitLab Personal Access Token. In Colab:
- Store tokens in Colab Secrets (key icon in left sidebar)
- Never commit tokens to the repository
- Use read-only tokens for exploration, write tokens only when needed

In [ ]:
# Configuration
import os
import json
import requests
from datetime import datetime, timedelta
from typing import Optional, Dict, List, Any

# Try to get token from Colab secrets, environment, or use placeholder
try:
    from google.colab import userdata
    GITLAB_TOKEN = userdata.get('GITLAB_TOKEN')
    print("✅ Token loaded from Colab Secrets")
except:
    GITLAB_TOKEN = os.getenv('GITLAB_TOKEN', 'YOUR_TOKEN_HERE')
    if GITLAB_TOKEN == 'YOUR_TOKEN_HERE':
        print("⚠️  No token found. Set GITLAB_TOKEN environment variable or use Colab Secrets.")
    else:
        print("✅ Token loaded from environment")

# Project configuration
GITLAB_URL = "https://gitlab.com/api/v4"
PROJECT_ID = 77260390  # CLARISSA (IRENA)

print(f"🔗 GitLab API: {GITLAB_URL}")
print(f"📁 Project ID: {PROJECT_ID}")
print(f"📅 Session: {datetime.now().isoformat()}")

In [ ]:
# GitLab API Client
class GitLabClient:
    """Simple client for GitLab CRUD operations."""
    
    def __init__(self, base_url: str, token: str, project_id: int):
        self.base_url = base_url.rstrip('/')
        self.project_id = project_id
        self.headers = {'PRIVATE-TOKEN': token}
    
    def _request(self, method: str, endpoint: str, data: Dict = None, params: Dict = None) -> Dict:
        """Make API request with error handling."""
        url = f"{self.base_url}/{endpoint.lstrip('/')}"
        try:
            response = requests.request(
                method=method,
                url=url,
                headers=self.headers,
                json=data,
                params=params,
                timeout=30
            )
            response.raise_for_status()
            return response.json() if response.text else {'status': 'success'}
        except requests.exceptions.HTTPError as e:
            return {'error': str(e), 'status_code': response.status_code, 'detail': response.text}
        except Exception as e:
            return {'error': str(e)}
    
    # ============ ISSUES ============
    def create_issue(self, title: str, description: str = None, labels: List[str] = None) -> Dict:
        """Create a new issue."""
        data = {'title': title}
        if description:
            data['description'] = description
        if labels:
            data['labels'] = ','.join(labels)
        return self._request('POST', f'projects/{self.project_id}/issues', data)
    
    def list_issues(self, state: str = 'opened', per_page: int = 10) -> List:
        """List issues."""
        return self._request('GET', f'projects/{self.project_id}/issues', 
                            params={'state': state, 'per_page': per_page})
    
    def get_issue(self, issue_iid: int) -> Dict:
        """Get issue details."""
        return self._request('GET', f'projects/{self.project_id}/issues/{issue_iid}')
    
    def update_issue(self, issue_iid: int, **kwargs) -> Dict:
        """Update issue (title, description, labels, state_event)."""
        return self._request('PUT', f'projects/{self.project_id}/issues/{issue_iid}', kwargs)
    
    def close_issue(self, issue_iid: int) -> Dict:
        """Close an issue."""
        return self.update_issue(issue_iid, state_event='close')
    
    def add_issue_comment(self, issue_iid: int, body: str) -> Dict:
        """Add comment to issue."""
        return self._request('POST', f'projects/{self.project_id}/issues/{issue_iid}/notes', {'body': body})
    
    def add_time_spent(self, issue_iid: int, duration: str) -> Dict:
        """Add time spent (e.g., '2h', '30m', '1h30m')."""
        return self._request('POST', f'projects/{self.project_id}/issues/{issue_iid}/add_spent_time',
                            {'duration': duration})
    
    # ============ MERGE REQUESTS ============
    def create_mr(self, source_branch: str, target_branch: str, title: str, description: str = None) -> Dict:
        """Create a merge request."""
        data = {
            'source_branch': source_branch,
            'target_branch': target_branch,
            'title': title
        }
        if description:
            data['description'] = description
        return self._request('POST', f'projects/{self.project_id}/merge_requests', data)
    
    def list_mrs(self, state: str = 'opened', per_page: int = 10) -> List:
        """List merge requests."""
        return self._request('GET', f'projects/{self.project_id}/merge_requests',
                            params={'state': state, 'per_page': per_page})
    
    def get_mr(self, mr_iid: int) -> Dict:
        """Get MR details."""
        return self._request('GET', f'projects/{self.project_id}/merge_requests/{mr_iid}')
    
    def merge_mr(self, mr_iid: int, squash: bool = False) -> Dict:
        """Merge a merge request."""
        return self._request('PUT', f'projects/{self.project_id}/merge_requests/{mr_iid}/merge',
                            {'squash': squash})
    
    # ============ PIPELINES ============
    def list_pipelines(self, per_page: int = 10, status: str = None) -> List:
        """List pipelines."""
        params = {'per_page': per_page}
        if status:
            params['status'] = status
        return self._request('GET', f'projects/{self.project_id}/pipelines', params=params)
    
    def get_pipeline(self, pipeline_id: int) -> Dict:
        """Get pipeline details."""
        return self._request('GET', f'projects/{self.project_id}/pipelines/{pipeline_id}')
    
    def get_pipeline_jobs(self, pipeline_id: int) -> List:
        """Get jobs for a pipeline."""
        return self._request('GET', f'projects/{self.project_id}/pipelines/{pipeline_id}/jobs')
    
    def trigger_pipeline(self, ref: str = 'main') -> Dict:
        """Trigger a new pipeline."""
        return self._request('POST', f'projects/{self.project_id}/pipeline', {'ref': ref})
    
    def cancel_pipeline(self, pipeline_id: int) -> Dict:
        """Cancel a running pipeline."""
        return self._request('POST', f'projects/{self.project_id}/pipelines/{pipeline_id}/cancel')
    
    def retry_pipeline(self, pipeline_id: int) -> Dict:
        """Retry failed jobs in a pipeline."""
        return self._request('POST', f'projects/{self.project_id}/pipelines/{pipeline_id}/retry')
    
    # ============ REPOSITORY ============
    def list_branches(self, per_page: int = 20) -> List:
        """List branches."""
        return self._request('GET', f'projects/{self.project_id}/repository/branches',
                            params={'per_page': per_page})
    
    def get_branch(self, branch_name: str) -> Dict:
        """Get branch details."""
        return self._request('GET', f'projects/{self.project_id}/repository/branches/{branch_name}')
    
    def create_branch(self, branch_name: str, ref: str = 'main') -> Dict:
        """Create a new branch."""
        return self._request('POST', f'projects/{self.project_id}/repository/branches',
                            {'branch': branch_name, 'ref': ref})
    
    def delete_branch(self, branch_name: str) -> Dict:
        """Delete a branch."""
        return self._request('DELETE', f'projects/{self.project_id}/repository/branches/{branch_name}')
    
    def list_commits(self, ref: str = 'main', per_page: int = 10) -> List:
        """List commits."""
        return self._request('GET', f'projects/{self.project_id}/repository/commits',
                            params={'ref_name': ref, 'per_page': per_page})
    
    def get_file(self, file_path: str, ref: str = 'main') -> Dict:
        """Get file content."""
        import urllib.parse
        encoded_path = urllib.parse.quote(file_path, safe='')
        return self._request('GET', f'projects/{self.project_id}/repository/files/{encoded_path}',
                            params={'ref': ref})
    
    def list_tree(self, path: str = '', ref: str = 'main') -> List:
        """List repository tree."""
        return self._request('GET', f'projects/{self.project_id}/repository/tree',
                            params={'path': path, 'ref': ref})
    
    # ============ LABELS ============
    def list_labels(self) -> List:
        """List project labels."""
        return self._request('GET', f'projects/{self.project_id}/labels')
    
    def create_label(self, name: str, color: str = '#428BCA', description: str = None) -> Dict:
        """Create a label."""
        data = {'name': name, 'color': color}
        if description:
            data['description'] = description
        return self._request('POST', f'projects/{self.project_id}/labels', data)
    
    # ============ PROJECT INFO ============
    def get_project(self) -> Dict:
        """Get project details."""
        return self._request('GET', f'projects/{self.project_id}')
    
    def get_runners(self) -> List:
        """List project runners."""
        return self._request('GET', f'projects/{self.project_id}/runners')

# Initialize client
gl = GitLabClient(GITLAB_URL, GITLAB_TOKEN, PROJECT_ID)
print("✅ GitLabClient initialized")

---

## 🏥 Health Check & Project Info

In [ ]:
# Get project info
project = gl.get_project()
if 'error' not in project:
    print(f"✅ Connected to: {project['name_with_namespace']}")
    print(f"   URL: {project['web_url']}")
    print(f"   Default branch: {project['default_branch']}")
    print(f"   Visibility: {project['visibility']}")
    print(f"   Open issues: {project.get('open_issues_count', 'N/A')}")
else:
    print(f"❌ Connection failed: {project}")

---

## 📋 Issues CRUD

### List Open Issues

In [ ]:
# READ: List issues
issues = gl.list_issues(state='opened', per_page=5)
print("📋 Open Issues:")
if isinstance(issues, list):
    for issue in issues:
        labels = ', '.join(issue.get('labels', [])) or 'none'
        print(f"  #{issue['iid']}: {issue['title'][:50]}... [{labels}]")
else:
    print(f"  {issues}")

### Create an Issue

In [ ]:
# CREATE: New issue
# Uncomment to create:

# new_issue = gl.create_issue(
#     title="[Playground] Test Issue",
#     description="Created from CRUD Playground notebook.\n\nThis is a test.",
#     labels=['test', 'playground']
# )
# print("📝 Created issue:")
# print(json.dumps(new_issue, indent=2))

print("⚠️ Issue creation is commented out. Uncomment to test.")

### Add Time Tracking

In [ ]:
# UPDATE: Add time spent to an issue
# Uncomment and set issue_iid:

# ISSUE_IID = 123  # Replace with actual issue number
# result = gl.add_time_spent(ISSUE_IID, '30m')
# print(f"⏱️ Added time: {result}")

print("⚠️ Time tracking is commented out. Uncomment to test.")

---

## 🚀 Pipelines CRUD

### List Recent Pipelines

In [ ]:
# READ: List pipelines
pipelines = gl.list_pipelines(per_page=5)
print("🚀 Recent Pipelines:")
if isinstance(pipelines, list):
    for p in pipelines:
        status_emoji = {'success': '✅', 'failed': '❌', 'running': '🔄', 'pending': '⏳'}.get(p['status'], '❓')
        print(f"  {status_emoji} #{p['id']}: {p['status']} on {p['ref']} ({p['created_at'][:10]})")
else:
    print(f"  {pipelines}")

### Get Pipeline Jobs

In [ ]:
# READ: Get jobs from latest pipeline
if isinstance(pipelines, list) and len(pipelines) > 0:
    latest = pipelines[0]
    jobs = gl.get_pipeline_jobs(latest['id'])
    print(f"📦 Jobs for pipeline #{latest['id']}:")
    if isinstance(jobs, list):
        for job in jobs:
            status_emoji = {'success': '✅', 'failed': '❌', 'running': '🔄'}.get(job['status'], '❓')
            duration = f"{job.get('duration', 0):.1f}s" if job.get('duration') else 'N/A'
            print(f"  {status_emoji} {job['name']}: {job['status']} ({duration})")
    else:
        print(f"  {jobs}")

### Trigger New Pipeline

In [ ]:
# CREATE: Trigger new pipeline
# Uncomment to trigger:

# new_pipeline = gl.trigger_pipeline(ref='main')
# print("🚀 Triggered pipeline:")
# print(json.dumps(new_pipeline, indent=2))

print("⚠️ Pipeline trigger is commented out. Uncomment to test.")

---

## 📁 Repository CRUD

### List Branches

In [ ]:
# READ: List branches
branches = gl.list_branches(per_page=10)
print("🌿 Branches:")
if isinstance(branches, list):
    for b in branches:
        protected = '🔒' if b.get('protected') else '  '
        print(f"  {protected} {b['name']}")
else:
    print(f"  {branches}")

### List Recent Commits

In [ ]:
# READ: List commits
commits = gl.list_commits(ref='main', per_page=5)
print("📝 Recent commits on main:")
if isinstance(commits, list):
    for c in commits:
        print(f"  {c['short_id']} - {c['title'][:60]}")
else:
    print(f"  {commits}")

### Browse Repository Tree

In [ ]:
# READ: List files in a directory
tree = gl.list_tree(path='docs/tutorials/notebooks')
print("📂 docs/tutorials/notebooks/:")
if isinstance(tree, list):
    for item in sorted(tree, key=lambda x: x['name']):
        icon = '📁' if item['type'] == 'tree' else '📄'
        print(f"  {icon} {item['name']}")
else:
    print(f"  {tree}")

### Create & Delete Branch

In [ ]:
# CREATE & DELETE: Branch lifecycle
# Uncomment to test:

# # Create
# new_branch = gl.create_branch('playground/test-branch', ref='main')
# print(f"🌿 Created: {new_branch}")

# # Delete
# deleted = gl.delete_branch('playground/test-branch')
# print(f"🗑️ Deleted: {deleted}")

print("⚠️ Branch operations are commented out. Uncomment to test.")

---

## 🏃 Runners Status

In [ ]:
# READ: List runners
runners = gl.get_runners()
print("🏃 Project Runners:")
if isinstance(runners, list):
    for r in runners:
        status = '🟢' if r.get('online') else '🔴'
        print(f"  {status} {r['description']} (ID: {r['id']})")
        print(f"      Tags: {', '.join(r.get('tag_list', []))}")
else:
    print(f"  {runners}")

---

## 🎮 Interactive Playground

In [ ]:
# 🎮 YOUR PLAYGROUND
# Try your own GitLab API calls here!

# Examples:
# gl.list_issues(state='closed', per_page=20)
# gl.get_issue(42)
# gl.list_mrs(state='merged')
# gl.get_file('README.md')

print("🎮 Edit this cell to try your own GitLab operations!")

In [ ]:
# 📋 Quick Reference
print("""
╔══════════════════════════════════════════════════════════════════╗
║                  GitLab API Quick Reference                      ║
╠══════════════════════════════════════════════════════════════════╣
║ ISSUES                                                           ║
║   gl.list_issues(state, per_page)      → List issues             ║
║   gl.create_issue(title, description)  → Create issue            ║
║   gl.update_issue(iid, **kwargs)       → Update issue            ║
║   gl.close_issue(iid)                  → Close issue             ║
║   gl.add_time_spent(iid, '2h')         → Track time              ║
╠══════════════════════════════════════════════════════════════════╣
║ MERGE REQUESTS                                                   ║
║   gl.list_mrs(state, per_page)         → List MRs                ║
║   gl.create_mr(source, target, title)  → Create MR               ║
║   gl.merge_mr(iid)                     → Merge MR                ║
╠══════════════════════════════════════════════════════════════════╣
║ PIPELINES                                                        ║
║   gl.list_pipelines(per_page, status)  → List pipelines          ║
║   gl.get_pipeline_jobs(pipeline_id)    → Get jobs                ║
║   gl.trigger_pipeline(ref)             → Trigger pipeline        ║
║   gl.cancel_pipeline(id)               → Cancel pipeline         ║
╠══════════════════════════════════════════════════════════════════╣
║ REPOSITORY                                                       ║
║   gl.list_branches()                   → List branches           ║
║   gl.list_commits(ref, per_page)       → List commits            ║
║   gl.list_tree(path)                   → Browse files            ║
║   gl.get_file(path)                    → Get file content        ║
╚══════════════════════════════════════════════════════════════════╝
""")

---

## 📚 Resources

- 📖 [GitLab API Documentation](https://docs.gitlab.com/ee/api/)
- 🔑 [Personal Access Tokens](https://docs.gitlab.com/ee/user/profile/personal_access_tokens.html)
- 🦊 [CLARISSA GitLab](https://gitlab.com/wolfram_laube/blauweiss_llc/irena)
- 🐙 [CLARISSA GitHub Mirror](https://github.com/wolfram-laube/clarissa)

---

*CLARISSA Portal - GitLab API Playground*